In [8]:
# Модели
from lightgbm import LGBMClassifier

# Пайплайн
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score

# Данные
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from category_encoders import BinaryEncoder, OneHotEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler, PowerTransformer, PolynomialFeatures, QuantileTransformer
from sklearn.utils.class_weight import compute_sample_weight

In [9]:
# Пути
ROOT = os.getcwd()
TRAIN_DATASET = os.path.join(ROOT, '../data/train_AIC.csv')
BALANCED_DATASET = os.path.join(ROOT, '../data/balanced_train.csv')
TEST_DATASET = os.path.join(ROOT, '../data/test_AIC.csv')
SUBMISSION_PATH = os.path.join(ROOT, '../submissions/')

def save_submission(model, subname):
    subname = os.path.join(SUBMISSION_PATH, f'{subname}.csv')
    preds = model.predict(test_df)
    submit_df = pd.DataFrame({'id': test_df.index, 'value': preds})
    submit_df.to_csv(subname, index=False)

# Загрузка
train_df = pd.read_csv(TRAIN_DATASET)
# balanced_df = pd.read_csv(BALANCED_DATASET, index_col=0)
# train_df = balanced_df
test_df = pd.read_csv(TEST_DATASET)

# first_negatives = train_df[train_df['y'] == 0][:train_df[train_df['y'] == 1]['y'].count()]
# train_df = pd.concat([train_df[train_df['y'] == 1], first_negatives])

# def random_undersample(df):
#     neg_count, pos_count = np.bincount(df['y'])
#     pos_df = df[df['y'] == 1]
#     neg_df = df[df['y'] == 0]
#     neg_df = neg_df.sample(n=pos_count, random_state=1708)
#     return pd.concat([pos_df, neg_df])

# balanced_df = random_undersample(train_df)

# Нормализация
# train_df = train_df[train_df['Длительность'] < 500]
# train_df = train_df[(train_df['Сумма'] > 2) & (train_df['Сумма'] < 10)]
# train_df = train_df[train_df['До поставки'] < 300]
# train_df = train_df[train_df['Дней между 0_1'] < 400]

X, y = train_df.iloc[:, :-1], train_df.iloc[:, -1]   
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train = pd.concat([X_train, y_train], axis=1)

# X_train = X_train[(X_train['Сумма'] > 2) & (X_train['Сумма'] < 10)]
# X_train = X_train[X_train['Длительность'] < 400]

y_train = X_train['y']
X_train = X_train.drop('y', axis=1)

In [10]:
# Категориальные фичи
cat_features = [
    'Purchasing Organization', 'Delivery Option', 
    'Company Code', 'Provider', 'Provider Delivery option', 
    'Provider Purchaser', 'Operations Manager', 'Sum Fold',
    'Month1', 'Month2', 'Month3', 'Weekday'
    ]

# Фичи на масштабирование
scale_features = [
    'Position Count', 'Duration', 'ETC Delivery',
    'Changes After Approvals', 'Order Approval 1', 'Order Approval 2',
    'Order Approval 3', 'Sum', 'Change Delivery Date 7', 'Change Delivery Date 15', 
    'Change Delivery Date 30', 'Approval Cycles', 'Handlers 7', 'Handlers 15', 
    'Handlers 30'
]

# Фичи на дроп
drop_features = [
    'Material', 'Cancel Complete Release', 'Amount',
    'Material Group', 'Delivery Date', 'Change on Paper', 
    'Category Manager', 'NRP', 'EI',
    'Days Between 0_1', 'Days Between 1_2', 'Days Between 2_3', 
    'Days Between 3_4', 'Days Between 4_5', 
    'Days Between 5_6', 'Days Between 6_7', 'Days Between 7_8'
    ]

# Новые имена фич
rename_cols = [
    'Provider', 'Material', 'Category Manager', 'Operations Manager',
    'Factory', 'Purchasing Organization', 'Purchasing Group', 
    'Company Code', 'EI', 'Material Group', 'Delivery Option', 'NRP',
    'Duration', 'ETC Delivery', 'Month1', 'Month2', 'Month3', 'Weekday',
    'Sum', 'Position Count', 'Amount', 'Handlers 7', 'Handlers 15', 
    'Handlers 30', 'Order Approval 1', 'Order Approval 2', 'Order Approval 3',
    'Change Delivery Date 7', 'Change Delivery Date 15', 'Change Delivery Date 30',
    'Cancel Complete Release', 'Change on Paper', 'Delivery Date', 
    'Approval Cycles', 'Changes After Approvals', 'Days Between 0_1', 
    'Days Between 1_2', 'Days Between 2_3', 'Days Between 3_4', 'Days Between 4_5', 
    'Days Between 5_6', 'Days Between 6_7', 'Days Between 7_8'
    ]

In [11]:
# Препроцессоры
class DataPreprocessor(BaseEstimator, TransformerMixin):
    """ Предобработчик данных """
    def __init__(self, cat_features, scale_features,
                 drop_features, rename_cols, transform_train=True):
        self.transform_train = transform_train
        self.cat_features = cat_features

        self.bin_encoder = BinaryEncoder(cols=cat_features)
        self.robust_scaler = RobustScaler()

        self.rename_cols = rename_cols

        self.drop_features = drop_features
        self.scale_features = scale_features

    def fit(self, X, y=None):
        # Создаём копию датасета
        X_ = X.copy()
        X_.columns = self.rename_cols

        # Временные фичи
        # X_['Weekday'] += 1
        # X_['day_sin'] = np.sin(np.pi * 2 * X_['Weekday'] / 7)
        # X_['day_cos'] = np.cos(np.pi * 2 * X_['Weekday'] / 7)
        # X_['month1_sin'] = np.sin(np.pi * 2 * X_['Month1'] / 12)
        # X_['month1_cos'] = np.cos(np.pi * 2 * X_['Month1'] / 12)
        # X_['month2_sin'] = np.sin(np.pi * 2 * X_['Month2'] / 12)
        # X_['month2_cos'] = np.cos(np.pi * 2 * X_['Month2'] / 12)
        # X_['month3_sin'] = np.sin(np.pi * 2 * X_['Month3'] / 12)
        # X_['month3_cos'] = np.cos(np.pi * 2 * X_['Month3'] / 12)

        # Экстракция фич
        X_['Provider Purchaser'] = [f'{x}_{y}' for x, y in zip(X_['Provider'].values, X_['Purchasing Organization'].values)]
        X_['Provider Delivery option'] = [f'{x}_{y}' for x, y in zip(X_['Provider'].values, X_['Delivery Option'].values)]
        X_['Sum Fold'] = X_['Sum'].apply(lambda x: int(x % 10))
        
        # Нормализация
        self.robust_scaler.fit(X_[self.scale_features])

        # Категориальные фичи        
        X_ = self.bin_encoder.fit_transform(X_)

        X_ = X_.drop(self.drop_features, axis=1)
        
        return self
    
    def transform(self, X):
        # Создаём копию датасета
        X_ = X.copy()
        X_.columns = self.rename_cols

        # Временные фичи
        # X_['Weekday'] += 1
        # X_['day_sin'] = np.sin(np.pi * 2 * X_['Weekday'] / 7)
        # X_['day_cos'] = np.cos(np.pi * 2 * X_['Weekday'] / 7)
        # X_['month1_sin'] = np.sin(np.pi * 2 * X_['Month1'] / 12)
        # X_['month1_cos'] = np.cos(np.pi * 2 * X_['Month1'] / 12)
        # X_['month2_sin'] = np.sin(np.pi * 2 * X_['Month2'] / 12)
        # X_['month2_cos'] = np.cos(np.pi * 2 * X_['Month2'] / 12)
        # X_['month3_sin'] = np.sin(np.pi * 2 * X_['Month3'] / 12)
        # X_['month3_cos'] = np.cos(np.pi * 2 * X_['Month3'] / 12)

        # Экстракция фич
        X_['Provider Purchaser'] = [f'{x}_{y}' for x, y in zip(X_['Provider'].values, X_['Purchasing Organization'].values)]
        X_['Provider Delivery option'] = [f'{x}_{y}' for x, y in zip(X_['Provider'].values, X_['Delivery Option'].values)]
        X_['Sum Fold'] = X_['Sum'].apply(lambda x: int(x % 10))

        # Нормализация
        X_[self.scale_features] = self.robust_scaler.transform(X_[self.scale_features])

        # Категориальные фичи
        X_ = self.bin_encoder.transform(X_)

        X_ = X_.drop(self.drop_features, axis=1)

        return X_


In [12]:
lgbm_params = [
    {
    'learning_rate': 0.3, 
    'n_estimators': 1000, 
    'max_depth': 21, 
    'max_bin': 160, 
    'num_leaves': 128,
    'reg_lambda': 0.2,
    },
    {
    'learning_rate': 0.3, 
    'n_estimators': 1500, 
    'max_depth': 21, 
    'max_bin': 160, 
    'num_leaves': 128,
    'reg_lambda': 0.2,
    },
    {
    'learning_rate': 0.3, 
    'n_estimators': 1500, 
    'max_depth': 16, 
    'max_bin': 160, 
    'num_leaves': 96,
    'reg_lambda': 0.2,
    },
    {
    'learning_rate': 0.3, 
    'n_estimators': 1500, 
    'max_depth': 21, 
    'max_bin': 128, 
    'num_leaves': 32,
    'reg_lambda': 0.2,
    },
    {
    'learning_rate': 0.3, 
    'n_estimators': 1500, 
    'max_depth': 12, 
    'max_bin': 256, 
    'num_leaves': 128,
    'reg_lambda': 0.2,
    }
]

models = []
for index, params in enumerate(lgbm_params):
    lgbm_model = LGBMClassifier(**params)
    models.append((f'lgbm_{index}', lgbm_model))

In [13]:
data_preprocessor = DataPreprocessor(cat_features, scale_features, drop_features, rename_cols)
stack = StackingClassifier(estimators=models, verbose=1, cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=1708), n_jobs=-1)
pipeline = Pipeline([
    ('data', data_preprocessor),
    ('model', stack)
])

In [20]:
pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)

In [23]:
print(f1_score(y_test, preds, average='macro'))
print(roc_auc_score(y_test, preds))

0.8985628003274775
0.875733539486157


In [16]:
print(f1_score(y_test, preds, average='macro'))
print(roc_auc_score(y_test, preds))

0.8985628003274775
0.875733539486157


In [17]:
# Для сравнения
print(f1_score(y_test, preds, average='macro'))
print(roc_auc_score(y_test, preds))

0.8985628003274775
0.875733539486157


In [18]:
feature_importances = pd.DataFrame({'Feature': pipeline['model'].feature_name_, 'Importance': pipeline['model'].feature_importances_})
feature_importances.to_csv('features.csv', index=False)

AttributeError: 'StackingClassifier' object has no attribute 'feature_name_'

In [ ]:
save_submission(pipeline, 'submission')